In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)


In [3]:
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [4]:
#Total Number of Players

uniqueplayer = purchase_data["SN"].unique()
uniqueplayer_count = len(uniqueplayer)
uniqueplayer_df = pd.DataFrame({"Total Players" : [uniqueplayer_count]})
uniqueplayer_df

,Total Players
0,576


In [5]:
# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

uniqueitems = purchase_data["Item ID"].nunique()
avgprice = purchase_data["Price"].mean()
totalpurchases = purchase_data["Purchase ID"].count()
sumrevenue = purchase_data["Price"].sum()

purchase_summary_df = pd.DataFrame({"Number of Unique Items" : [uniqueitems],
                                   "Average Purchase Price" : ["${:,.2f}".format(avgprice)],
                                   "Total Number of Purchases" : [totalpurchases],
                                   "Total Revenue" : ["${:,.2f}".format(sumrevenue)]})
purchase_summary_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [18]:
# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

wholecount = purchase_data["Gender"].count()
unique_df = purchase_data.groupby(["Gender"]).count()

unique_df["Percentage of Players"] = (unique_df["Purchase ID"] / wholecount) * 100
def format(x):
        return "{:.2f}%".format(x)
unique_df["Percentage of Players"] = unique_df["Percentage of Players"].apply(format)

unique_df = unique_df.drop(columns={"SN","Age","Item ID","Item Name","Price"})
unique_df = unique_df.rename(columns={"Purchase ID":"Total Count of Players"})
unique_df

,Total Count of Players,Percentage of Players
Gender,,
Female,113,14.49%
Male,652,83.59%
Other / Non-Disclosed,15,1.92%


In [25]:
# The below each broken by gender
gender_needed = pd.DataFrame(purchase_data, columns=["SN","Gender","Purchase ID","Price"])

# Purchase Count
purcount_gen = gender_needed.groupby(["Gender"]).count()
# Average Purchase Price
avgprice_gen = gender_needed.groupby(["Gender"]).mean()
# Total Purchase Value
tvp_gen = gender_needed.groupby(["Gender"]).sum()

merge_gen = pd.DataFrame.merge(purcount_gen, avgprice_gen, on="Gender",suffixes=("_count","_avg"))
merge_gen2 = pd.DataFrame.merge(merge_gen, tvp_gen, on="Gender",suffixes=("_merge","_tvp"))

merge_gen2 = merge_gen2.drop(columns={"Purchase ID_count","Price_count","Purchase ID_avg","Purchase ID"})
merge_gen2 = merge_gen2.rename(columns={"SN":"Purchase Count","Price_avg":"Average Purchase Price","Price":"Total Purchase Value"})
def format(x):
        return "${:.2f}".format(x)
merge_gen2["Average Purchase Price"] = merge_gen2["Average Purchase Price"].apply(format)
merge_gen2["Total Purchase Value"] = merge_gen2["Total Purchase Value"].apply(format)
merge_gen2
# Average Purchase Total per Person by Gender



,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,$3.20,$361.94
Male,652,$3.02,$1967.64
Other / Non-Disclosed,15,$3.35,$50.19


In [70]:
a = purchase_data["Age"].max()
b = purchase_data["Age"].min()
a , b

(45, 7)

In [8]:
# The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

# Purchase Count
# Average Purchase Price
# Total Purchase Value
# Average Purchase Total per Person by Age Group
age_needed = pd.DataFrame(purchase_data, columns=["SN","Age","Purchase ID","Price"])
unique_age = age_needed.groupby("SN").mean()

great = unique_age["Age"].max()
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, great]

group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]

unique_age["Age Bracket"] = pd.cut(unique_age["Age"], bins, labels=group_names, include_lowest=True)
wholecount = unique_age["Age"].count()

unique_age = unique_age.groupby("Age Bracket").count()

unique_age["Percentage of Players"] = (unique_age["Age"] / wholecount) * 100
def format(x):
        return "{:.2f}%".format(x)
unique_age["Percentage of Players"] = unique_age["Percentage of Players"].apply(format)
unique_age = unique_age.drop(columns={"Purchase ID","Price"})

unique_age

,Age,Percentage of Players
Age Bracket,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [184]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
player_needitem_df = pd.DataFrame(purchase_data, columns=["SN","Purchase ID","Price"])

player_purchase_count = player_needitem_df.groupby("SN").count()
player_tvp = player_needitem_df.groupby("SN").sum()
player_average = player_needitem_df.groupby("SN").mean()

player_merge1 = pd.DataFrame.merge(player_purchase_count, player_tvp, on="SN", suffixes=("_count","_tvp"))
player_merge2 = pd.DataFrame.merge(player_merge1, player_average, on="SN", suffixes=("","_average"))
player_merge2 = player_merge2.drop(columns={"Purchase ID_tvp","Price_count","Purchase ID"})
player_merge2 = player_merge2.rename(columns={"Purchase ID_count":"Purchase Count","Price":"Average Purchase Price","Price_tvp":"Total Purchase Value"})

player_merge2 = player_merge2.sort_values("Total Purchase Value", ascending=False)

player_merge2 = player_merge2[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

def format(x):
        return "${:.2f}".format(x)
player_merge2['Average Purchase Price'] = player_merge2['Average Purchase Price'].apply(format)
player_merge2['Total Purchase Value'] = player_merge2['Total Purchase Value'].apply(format)

player_merge2

# SN
# Purchase Count
# Average Purchase Price
# Total Purchase Value



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [222]:
# Identify the 5 most popular items by purchase count, then list (in a table):
popneeditem_df = pd.DataFrame(purchase_data, columns=["Item ID","Item Name","Price"])

pop_tvp = popneeditem_df.groupby(["Item ID","Item Name"]).sum()
pop_item_purcount = popneeditem_df.groupby(["Item ID","Item Name"]).count()
pop_itemprice = popneeditem_df.groupby(["Item ID","Item Name"]).mean()

pop_merge1 = pd.DataFrame.merge(pop_tvp, pop_item_purcount, on=["Item ID","Item Name"])
pop_merge2 = pd.DataFrame.merge(pop_merge1, pop_itemprice, on=["Item ID","Item Name"])
pop_merge2 = pop_merge2.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Purchase Count","Price":"Item Price"})
pop_merge2 = pop_merge2[["Purchase Count","Item Price", "Total Purchase Value"]]

pop_merge2 = pop_merge2.sort_values("Purchase Count", ascending=False)

def format(x):
    return "${:.2f}".format(x)

pop_merge2['Total Purchase Value'] = pop_merge2['Total Purchase Value'].apply(format)
pop_merge2['Item Price'] = pop_merge2['Item Price'].apply(format)

pop_merge2.head(5)


# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [220]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):
needitem_df = pd.DataFrame(purchase_data, columns=["Item ID","Item Name","Price"])

tvp = needitem_df.groupby(["Item ID","Item Name"]).sum()
item_purcount = needitem_df.groupby(["Item ID","Item Name"]).count()
itemprice = needitem_df.groupby(["Item ID","Item Name"]).mean()

merge1 = pd.DataFrame.merge(tvp, item_purcount, on=["Item ID","Item Name"])
merge2 = pd.DataFrame.merge(merge1, itemprice, on=["Item ID","Item Name"])
merge2 = merge2.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Purchase Count","Price":"Item Price"})
merge2 = merge2[["Purchase Count","Item Price", "Total Purchase Value"]]

merge2 = merge2.sort_values("Total Purchase Value", ascending=False)

def format(x):
    return "${:.2f}".format(x)

merge2['Total Purchase Value'] = merge2['Total Purchase Value'].apply(format)
merge2['Item Price'] = merge2['Item Price'].apply(format)

merge2.head(5)


# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
